# LIAR Fake News Detection

# Setup

## Install Libraries

In [1]:
%pip install -r requirements.txt
!spacy download en_core_web_sm

Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 9.8 MB/s eta 0:00:00ta 0:00:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


## Import Libraries

In [2]:
import nltk
import spacy
import pandas as pd
import tensorflow as tf
# from tensorflow_utils import get_info
# get_info.tf_info()
# get_info.gpu_info()
nltk.download('stopwords')
nlp = spacy.load('en_core_web_sm')
# get the GPU device name
tf.config.list_physical_devices()

2024-11-22 00:12:52.322607: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-22 00:12:52.483717: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1732214572.542549   19349 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1732214572.562132   19349 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-22 00:12:52.737256: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

## Load Data

In [3]:
train_data = pd.read_table('Data/train.tsv', names = ["id", "label", "statement", "subject", "speaker", "job", "state", "party", "barely-true", "false", "half-true", "mostly-true", "pants-fire", "venue"])
test_data = pd.read_table('Data/test.tsv', names = ["id", "label", "statement", "subject", "speaker", "job", "state", "party", "barely-true", "false", "half-true", "mostly-true", "pants-fire", "venue"])
valid_data = pd.read_table('Data/valid.tsv', names = ["id", "label", "statement", "subject", "speaker", "job", "state", "party", "barely-true", "false", "half-true", "mostly-true", "pants-fire", "venue"])

### Data Info

In [7]:
print("Training Data Info:")
print(train_data.info())
print("Testing Data Info:")
print(test_data.info())
print("Validation Data Info:")
print(valid_data.info())
print(train_data.label.unique())
print(train_data.head())

Training Data Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10240 entries, 0 to 10239
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           10240 non-null  object 
 1   label        10240 non-null  object 
 2   statement    10240 non-null  object 
 3   subject      10238 non-null  object 
 4   speaker      10238 non-null  object 
 5   job          7342 non-null   object 
 6   state        8030 non-null   object 
 7   party        10238 non-null  object 
 8   barely-true  10238 non-null  float64
 9   false        10238 non-null  float64
 10  half-true    10238 non-null  float64
 11  mostly-true  10238 non-null  float64
 12  pants-fire   10238 non-null  float64
 13  venue        10138 non-null  object 
dtypes: float64(5), object(9)
memory usage: 1.1+ MB
None
Testing Data Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1267 entries, 0 to 1266
Data columns (total 14 columns):
 #   Column     

# Preprocessing

## Convert Categorical Data to Numerical

### Output Labels

In [ ]:
# based on degree of truthfullness
y_label_dict = {"pants-fire" : 0, "false" : 1, "barely-true" : 2, "half-true" : 3, "mostly-true" : 4, "true" : 5}

train_data['output'] = train_data['label'].apply(lambda i: y_label_dict[i])
valid_data['output'] = valid_data['label'].apply(lambda i: y_label_dict[i])
test_data['output'] = test_data['label'].apply(lambda i: y_label_dict[i])


### Speakers

In [29]:
# take number of top speakers to consider a parameter
no_speaker = 25
# based on the frequency of the label (only consider the top no_speaker speakers as relevent, after 20, rest have less than 50 data points, not relevent)
frequent_speakers = train_data['speaker'].value_counts().reset_index()[:no_speaker].to_dict()['speaker']
frequent_speakers = dict((v, k) for k, v in frequent_speakers.items())
print(frequent_speakers)

def convert_speaker_to_num(speaker):
  # speaker not in the top 20, assign it to the 21st category
  other = no_speaker
  if isinstance(speaker, str):
    if speaker in frequent_speakers:
      return frequent_speakers[speaker]
    else:
      return other
  else:
    return other

train_data['speaker_id'] = train_data['speaker'].apply(convert_speaker_to_num)
valid_data['speaker_id'] = valid_data['speaker'].apply(convert_speaker_to_num)
test_data['speaker_id'] = test_data['speaker'].apply(convert_speaker_to_num)
train_data['speaker_id'].value_counts()

{'barack-obama': 0, 'donald-trump': 1, 'hillary-clinton': 2, 'mitt-romney': 3, 'scott-walker': 4, 'john-mccain': 5, 'rick-perry': 6, 'chain-email': 7, 'marco-rubio': 8, 'rick-scott': 9, 'ted-cruz': 10, 'bernie-s': 11, 'chris-christie': 12, 'facebook-posts': 13, 'charlie-crist': 14, 'newt-gingrich': 15, 'jeb-bush': 16, 'joe-biden': 17, 'blog-posting': 18, 'paul-ryan': 19, 'sarah-palin': 20, 'john-boehner': 21, 'michele-bachmann': 22, 'rick-santorum': 23, 'national-republican-congressional-committee': 24}


speaker_id
25    7295
0      488
1      273
2      239
3      176
4      149
5      148
7      142
6      142
8      117
9      115
10      93
11      88
13      78
12      78
14      70
15      69
17      63
16      63
18      59
19      56
20      52
21      49
24      46
22      46
23      46
Name: count, dtype: int64

### Job Titles

In [30]:
# take number of top jobs to consider a parameter
no_jobs = 25
# based on the frequency of the label (only consider the top no_jobs speakers as relevent, after 20, rest have less than 50 data points, not relevent)
frequent_jobs = train_data['job'].value_counts().reset_index()[:no_jobs].to_dict()['job']
frequent_jobs = dict((v, k) for k, v in frequent_jobs.items())
print(frequent_jobs)

def convert_job_to_num(job):
  # job not in the top jobs, assign it to the last category
  other = no_jobs
  if isinstance(job, str):
    if job in frequent_jobs:
      return frequent_jobs[job]
    else:
      return other
  else:
    return other

train_data['job_id'] = train_data['job'].apply(convert_job_to_num)
valid_data['job_id'] = valid_data['job'].apply(convert_job_to_num)
test_data['job_id'] = test_data['job'].apply(convert_job_to_num)
train_data['job_id'].value_counts()

{'President': 0, 'U.S. Senator': 1, 'Governor': 2, 'President-Elect': 3, 'U.S. senator': 4, 'Presidential candidate': 5, 'Former governor': 6, 'U.S. Representative': 7, 'Milwaukee County Executive': 8, 'Senator': 9, 'State Senator': 10, 'U.S. representative': 11, 'U.S. House of Representatives': 12, 'Attorney': 13, 'Congressman': 14, 'Social media posting': 15, 'Governor of New Jersey': 16, 'Co-host on CNN\'s "Crossfire"': 17, 'State Representative': 18, 'State representative': 19, 'U.S. Congressman': 20, 'Congresswoman': 21, 'Speaker of the House of Representatives': 22, 'State senator': 23, 'state representative': 24}


job_id
25    6348
0      492
1      479
2      391
3      273
4      263
5      254
6      176
7      172
8      149
9      147
10     108
11     103
12     102
13      81
14      80
15      78
16      78
17      73
18      72
19      66
20      63
22      50
21      50
23      48
24      44
Name: count, dtype: int64

### Parties

In [32]:
# take number of top parties to consider a parameter
no_party = 9
# based on the frequency of the label (only consider the top no_party speakers as relevent, after 20, rest have less than 50 data points, not relevent)
frequent_party = train_data['party'].value_counts().reset_index()[:no_party].to_dict()['party']
frequent_party = dict((v, k) for k, v in frequent_party.items())
print(frequent_party)

def convert_party_to_num(party):
  # party not in the top parties, assign it to the last category
  other = no_party
  if isinstance(party, str):
    if party in frequent_party:
      return frequent_party[party]
    else:
      return other
  else:
    return other

train_data['party_id'] = train_data['party'].apply(convert_party_to_num)
valid_data['party_id'] = valid_data['party'].apply(convert_party_to_num)
test_data['party_id'] = test_data['party'].apply(convert_party_to_num)
train_data['party_id'].value_counts()

{'republican': 0, 'democrat': 1, 'none': 2, 'organization': 3, 'independent': 4, 'newsmaker': 5, 'libertarian': 6, 'activist': 7, 'journalist': 8}


party_id
0    4497
1    3336
2    1744
3     219
4     147
9     124
5      56
6      40
7      39
8      38
Name: count, dtype: int64

### States

In [35]:
# take number of top states to consider a parameter
no_state = 30
# based on the frequency of the label (only consider the top no_state speakers as relevent, after 20, rest have less than 50 data points, not relevent)
frequent_state = train_data['state'].value_counts().reset_index()[:no_state].to_dict()['state']
frequent_state = dict((v, k) for k, v in frequent_state.items())
print(frequent_state)

def convert_state_to_num(state):
  # state not in the top states, assign it to the last category
  other = no_state
  if isinstance(state, str):
    if state in frequent_state:
      return frequent_state[state]
    else:
      return other
  else:
    return other

train_data['state_id'] = train_data['state'].apply(convert_state_to_num)
valid_data['state_id'] = valid_data['state'].apply(convert_state_to_num)
test_data['state_id'] = test_data['state'].apply(convert_state_to_num)
train_data['state_id'].value_counts()

{'Texas': 0, 'Florida': 1, 'Wisconsin': 2, 'New York': 3, 'Illinois': 4, 'Ohio': 5, 'Georgia': 6, 'Virginia': 7, 'Rhode Island': 8, 'New Jersey': 9, 'Oregon': 10, 'Massachusetts': 11, 'Arizona': 12, 'California': 13, 'Washington, D.C.': 14, 'Vermont': 15, 'Pennsylvania': 16, 'New Hampshire': 17, 'Arkansas': 18, 'Tennessee': 19, 'Kentucky': 20, 'Maryland': 21, 'Delaware': 22, 'Alaska': 23, 'Minnesota': 24, 'North Carolina': 25, 'Nevada': 26, 'Indiana': 27, 'Missouri': 28, 'New Mexico': 29}


state_id
30    2539
0     1009
1      997
2      713
3      657
4      556
5      447
6      426
7      407
8      369
9      241
10     239
11     206
12     182
13     159
14     120
15      98
16      90
17      86
18      84
19      75
20      74
21      69
22      68
23      65
24      56
25      56
26      48
27      38
28      36
29      30
Name: count, dtype: int64

### Subjects

In [37]:
# take number of top subjects to consider a parameter
no_subject = 30
# based on the frequency of the label (only consider the top no_subject speakers as relevent, after 20, rest have less than 50 data points, not relevent)
frequent_subject = train_data['subject'].value_counts().reset_index()[:no_subject].to_dict()['subject']
frequent_subject = dict((v, k) for k, v in frequent_subject.items())
print(frequent_subject)

def convert_subject_to_num(subject):
  # subject not in the top subjects, assign it to the last category
  other = no_subject
  if isinstance(subject, str):
    if subject in frequent_subject:
      return frequent_subject[subject]
    else:
      return other
  else:
    return other

train_data['subject_id'] = train_data['subject'].apply(convert_subject_to_num)
valid_data['subject_id'] = valid_data['subject'].apply(convert_subject_to_num)
test_data['subject_id'] = test_data['subject'].apply(convert_subject_to_num)
train_data['subject_id'].value_counts()

{'health-care': 0, 'taxes': 1, 'immigration': 2, 'elections': 3, 'education': 4, 'candidates-biography': 5, 'economy': 6, 'guns': 7, 'economy,jobs': 8, 'federal-budget': 9, 'jobs': 10, 'energy': 11, 'abortion': 12, 'foreign-policy': 13, 'state-budget': 14, 'education,state-budget': 15, 'transportation': 16, 'crime': 17, 'ethics': 18, 'iraq': 19, 'campaign-finance': 20, 'terrorism': 21, 'environment': 22, 'history': 23, 'job-accomplishments': 24, 'legal-issues': 25, 'social-security': 26, 'deficit,federal-budget': 27, 'state-budget,taxes': 28, 'energy,environment': 29}


subject_id
30    6910
0      381
1      308
2      253
3      252
4      237
5      190
6      137
7      130
8      125
9      121
10      98
11      94
12      92
13      85
14      75
15      69
16      64
17      59
18      58
19      55
20      53
21      53
22      52
24      45
23      45
25      42
27      40
26      40
28      39
29      38
Name: count, dtype: int64

### Venues

In [ ]:
# take number of top venues to consider a parameter
no_venue = 30
# based on the frequency of the label (only consider the top no_venue speakers as relevent, after 20, rest have less than 50 data points, not relevent)
frequent_venue = train_data['venue'].value_counts().reset_index()[:no_venue].to_dict()['venue']
frequent_venue = dict((v, k) for k, v in frequent_venue.items())
print(frequent_venue)

def convert_venue_to_num(venue):
  # venue not in the top venues, assign it to the last category
  other = no_venue
  if isinstance(venue, str):
    if venue in frequent_venue:
      return frequent_venue[venue]
    else:
      return other
  else:
    return other

train_data['venue_id'] = train_data['venue'].apply(convert_venue_to_num)
valid_data['venue_id'] = valid_data['venue'].apply(convert_venue_to_num)
test_data['venue_id'] = test_data['venue'].apply(convert_venue_to_num)
train_data['venue_id'].value_counts()

## Tokenizing Content

### Statements

In [ ]:
#TODO